<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [15]:
import os
import warnings
warnings.filterwarnings('ignore')
import re
from tqdm import tqdm
import pandas as pd
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import urllib

In [21]:
df[df.title.str.contains('babyl')]

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId
4116,James McKay,Babylon,12/0,ENTP,XwX,movies,https://static1.personality-database.com/profi...,168,babylon,86657.0
10739,Nellie LaRoy,Babylon,57/1,ESFP,7w8,movies,https://static1.personality-database.com/profi...,46,babylon,86657.0
13147,Jack Conrad,Babylon,38/1,ESTP,3w2,movies,https://static1.personality-database.com/profi...,66,babylon,86657.0
15347,Manny Torres,Babylon,35/2,INFJ,9w1,movies,https://static1.personality-database.com/profi...,65,babylon,86657.0
15357,Manny Torres,Babylon,35/2,INFJ,9w1,movies,https://static1.personality-database.com/profi...,67,babylon,86657.0


In [16]:
df = pd.read_csv('../MBTI_merge_movieLens_3229_movie.csv')
movie = df[['title','movieId']].drop_duplicates('movieId')
movie = movie.reset_index(drop=True)
movie

,title,movieId
0,moana,73141.0
1,inside out,122470.0
2,zootopia,152081.0
3,legally blonde,4447.0
4,dead poets society,1246.0
...,...,...
3224,cube²: hypercube,27482.0
3225,the cremator,76208.0
3226,dark waters,177115.0
3227,the young victoria,68269.0


In [17]:
movie[movie.title.str.contains('mohabba')]

,title,movieId
680,mohabbatein,74426.0


In [18]:
custom_movie = movie[681:829]
custom_movie

,title,movieId
681,enchanted,56152.0
682,treasure island,6405.0
683,alpha and omega,88468.0
684,in time,90405.0
685,o auto da compadecida,27423.0
...,...,...
824,signs,5502.0
825,repulsion,3075.0
826,finding neverland,8970.0
827,the brave little toaster,4090.0


In [19]:
driver = webdriver.Chrome(ChromeDriverManager().install())

directory="custom_movie_images"
os.makedirs(directory,exist_ok=True)
timeout=120 # element가 화면에 뜰때까지 기다리는 시간

for l in tqdm(custom_movie.itertuples(), total=custom_movie.shape[0]):
    cleaned_title = re.sub(r'[^\w\s]', '', l.title)
    result_title="_".join(cleaned_title.split())
    custom_movieid = int(l.movieId)
    filename = f"{custom_movieid}_{result_title}"
    
    search_query=cleaned_title+' 영화 포스터 위키'
    URL=f"https://www.google.co.kr/search?q={search_query}&hl=ko&tbm=isch"
    driver.get(URL)
    
    #islrg > div.islrc > div:nth-child(2) > a.wXeWr.islib.nfEiy > div.bRMDJf.islir > img
    css_selector=".wXeWr.islib.nfEiy > .bRMDJf.islir > img"
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)) # 프로필 카드
    img = WebDriverWait(driver, timeout).until(element_present) # 프로필 카드가 모두 있을 때까지 Wait
    img_src = img.get_attribute("src")
    urllib.request.urlretrieve(img_src, f"{directory}/{filename}.png")
driver.close()

100%|██████████| 148/148 [03:13<00:00,  1.31s/it]


In [14]:
driver.close()